Übung:  09-2 Ingress
-------------------

![](demo/images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Statt des Apache-Webservers, der als [Reverse Proxy](https://github.com/ewolff/microservice-kubernetes/blob/master/microservice-kubernetes-demo/apache/000-default.conf) konfiguriert ist, wird die Kubernetes Ressource Ingress verwendet.

In [ ]:
# ! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml (obsolet!)
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

Nach dem Starten der Services erstellen wir die Ingress Ressourcen:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /\$2
  name: order
  namespace: ms-kubernetes 
  labels:
    app: order
spec:
  rules:
  - http:
      paths:
      - path: /order/
        backend:
          serviceName: order
          servicePort: 8080
---
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: catalog
  namespace: ms-kubernetes 
  labels:
    app: catalog
spec:
  rules:
  - http:
      paths:
      - path: /catalog/
        backend:
          serviceName: catalog
          servicePort: 8080
---
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: customer 
  namespace: ms-kubernetes 
  labels:
    app: customer 
spec:
  rules:
  - http:
      paths:
      - path: /customer 
        backend:
          serviceName: customer 
          servicePort: 8080   
%EOF%


Überprüfen der erstellen Ressourcen

In [ ]:
! kubectl get all,ingress -n ms-kubernetes

Visuell lässt sich das Resultat durch Anwählen der URLs verifizieren (Port 30443 = ngnix):

In [ ]:
! echo "Order   : $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e "s/6443/30443/")/order/list.html"
! echo "Catalog : $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e "s/6443/30443/")/catalog/"
! echo "Customer: $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e "s/6443/30443/")/customer/"

- - -

Aufräumen

In [ ]:
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/hystrix.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

***
Fragen
======

Beantwortet die Fragen einzeln oder in der Gruppe und vergleicht diese mit den Antworten


Was ein Ingress?
<details><summary>Antwort</summary><p>    
    Ein API-Objekt, das den externen Zugriff auf die Dienste in einem Cluster verwaltet, in der Regel mittels HTTP. 
    Grob entspricht der Ingress Dienst dem Reverse Proxy Muster. 
</p></details>

---

Was für eine Netzwerkstruktur verwendet Kubernetes?
<details><summary>Antwort</summary><p>    
     Kubernetes verwendet im Unterschied zu Docker eine flache Netzwerkstruktur. 
* Jeder Container kann mit jedem anderen ohne NAT kommunizieren.
* Alle Kubernetes Nodes können mit allen Containern (und in die andere Richtung) ohne NAT kommunizieren.
* Die IP, die ein Container von sich selbst sieht, ist auch die, die jeder andere Node oder Container im Netz von ihm sieht.
</p></details>

---